In [1]:
import sys
sys.path.append('../src')
# from causal_shapley import causal_shapley, predict_proba
from egtoolkit import *
%load_ext autoreload
%autoreload 2

from egtoolkit import *

import random
import warnings
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import itertools
from itertools import combinations, permutations, chain

import shap

warnings.filterwarnings("ignore")
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.under_sampling import RandomUnderSampler

from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete.CPD import TabularCPD

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


## Import dataset

In [2]:
categorical_cols = ['global_active_power', 'global_reactive_power', 'voltage',
                    'global_intensity', 'kitchen', 'laundry', 'climate_control', 'other',
                    'weekend', 'month_name', 'season_name', 'day_name']  # list all columns that are categorical

In [3]:
dataset = pd.read_csv('./datasets/2.0-discretized-v2-3-peak-label-encoded.csv')

In [4]:
dataset[['peak_warning']].values

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [5]:
targets = ['peak_label_pred', 'peak_warning', 'no_significant_change', 'lower_than_usual']
targets_only = ['peak_label_pred', 'no_significant_change', 'lower_than_usual']

In [6]:
dataset.columns

Index(['global_active_power', 'global_reactive_power', 'voltage',
       'global_intensity', 'kitchen', 'laundry', 'climate_control', 'other',
       'weekend', 'month_name', 'season_name', 'day_name', 'peak_label_pred',
       'peak_warning', 'no_significant_change', 'lower_than_usual'],
      dtype='object')

In [7]:
dataset['peak_warning'].values

array([False, False, False, ..., False, False, False])

In [8]:
dataset.shape

(41673, 16)

In [9]:
sample = 20
background = dataset.drop(columns=targets).sample(sample, random_state=4)
indexes = background.index.tolist()
dataset.iloc[indexes]

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,peak_warning,no_significant_change,lower_than_usual
38128,Low,Medium,High,Low,Medium,Medium,Medium,Low,True,september,autumn,sunday,lower_than_usual,False,False,True
34353,Medium,Low,Medium,Medium,Medium,Low,High,Low,False,september,autumn,tuesday,lower_than_usual,False,False,True
24951,Very High,High,Low,Very High,Low,Very High,High,Very High,True,april,spring,saturday,peak_warning,True,False,False
32779,Low,Low,High,Low,Low,Low,Low,Medium,False,january,winter,monday,lower_than_usual,False,False,True
37580,Low,Medium,Low,Low,Low,Low,Low,Low,False,july,summer,friday,lower_than_usual,False,False,True
14887,Very High,Very High,Low,Very High,Very High,Very High,Very High,Very High,True,october,autumn,sunday,peak_warning,True,False,False
4375,Medium,Medium,High,Medium,Medium,Low,Low,High,False,march,spring,tuesday,no_significant_change,False,True,False
12741,Low,Low,Medium,Low,Very High,High,High,Low,False,september,autumn,thursday,no_significant_change,False,True,False
27682,High,High,Low,High,Very High,High,Very High,Medium,False,may,spring,friday,peak_warning,True,False,False
8848,Very High,Low,High,Very High,Medium,High,Very High,Very High,False,january,winter,wednesday,no_significant_change,False,True,False


In [10]:
_data = dataset.drop(columns=targets_only)

def find_outcome(observation):
    # Iterate through each record in the data
    # print(len(data.values))
    for record in _data.values:
        # print('obs', observation)
        if (record[:-1] == observation).all():
            # print('Found!')
            # print('Record: ', record)
            # print('Observation: ', observation)
            return record[-1]  # Return the last item (True/False) if there's a match
    return None  # Return None if  no match is found

def custom_predict(data):
    # Debugging to check input data shape
    print('=================')
    # print("Input shape to custom_predict:", data.shape)
    # if data.shape[0] == 621600:
    #     print(data)

    # Assuming boolean to integer conversion as before
    predictions = []
    # encode = {True: 1.0, False: 0.0}
    encode = {True: 1.0, False: 0.0, None: random.uniform(0.4, 0.6)}
    if isinstance(data, pd.DataFrame):
        print('dataframe instance')
        predictions = data['peak_warning'].apply(lambda x: encode[x]).values
    else:
        print('non dataframe instance')
        # predictions = np.array([encode[bool(p)] for p in data[:, -1]])
        # print(predictions)
        # print(p)
        print(f'Finding outcome based on prediction result with data Shape {data.shape}...')
        for observation in data[:,:]:
            predictions.append(encode[find_outcome(observation)])
            # print(predictions)
            # break
        # print(predictions)
        predictions = np.array(predictions)
        
    # print('data', data)
    
    # Ensure the output is correctly shaped
    predictions = predictions.reshape(-1, 1)
    print("Output shape from custom_predict:", predictions.shape)
    print('Done...')
    print('')
    return predictions
explainer = shap.KernelExplainer(custom_predict, background)

non dataframe instance
Finding outcome based on prediction result with data Shape (20, 12)...
Output shape from custom_predict: (20, 1)
Done...



In [11]:
observation = dataset.drop(columns=targets).iloc[0:1]
print("Observation shape before SHAP:", observation.shape)

try:
    shap_values = explainer.shap_values(observation)
    # shap_values = explainer.shap_values(background.iloc[0])
except Exception as e:
    print("Error during SHAP computation:", e)
# (10360, 1)

Observation shape before SHAP: (1, 12)


  0%|          | 0/1 [00:00<?, ?it/s]INFO:shap:num_full_subsets = 3
INFO:shap:remaining_weight_vector = array([0.35673839, 0.32616082, 0.31710079])
INFO:shap:num_paired_subset_sizes = 5
INFO:shap:weight_left = 0.2929005745959313


non dataframe instance
Finding outcome based on prediction result with data Shape (1, 12)...
Output shape from custom_predict: (1, 1)
Done...

non dataframe instance
Finding outcome based on prediction result with data Shape (41440, 12)...


INFO:shap:phi = array([-0.01781944, -0.08963618, -0.00474201, -0.01406532, -0.04127581,
       -0.04398404, -0.03596046, -0.00663886, -0.02190608, -0.05162666,
       -0.03436907, -0.03797608])
100%|██████████| 1/1 [1:07:37<00:00, 4057.80s/it]

Output shape from custom_predict: (41440, 1)
Done...



In [ ]:
pd.DataFrame()

In [20]:
# peak condition
observation_peak = pd.DataFrame([{
    'global_active_power': 'Low',
    'global_reactive_power': 'Medium',
    'voltage': 'High',
    'global_intensity': 'Low',
    'kitchen': 'Medium',
    'laundry': 'Medium',
    'climate_control': 'Medium',
    'other': 'Low',
    'weekend': 'True',
    'month_name': 'september',
    'season_name': 'autumn',
    'day_name': 'sunday'
}])
observation_peak

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name
0,## value here,## value here,## value here,## value here,## value here,## value here,## value here,## value here,## value here,## value here,## value here,## value here


In [12]:
background.iloc[0], shap_values

(global_active_power            Low
 global_reactive_power       Medium
 voltage                       High
 global_intensity               Low
 kitchen                     Medium
 laundry                     Medium
 climate_control             Medium
 other                          Low
 weekend                       True
 month_name               september
 season_name                 autumn
 day_name                    sunday
 Name: 38128, dtype: object,
 [array([[-0.01781944, -0.08963618, -0.00474201, -0.01406532, -0.04127581,
          -0.04398404, -0.03596046, -0.00663886, -0.02190608, -0.05162666,
          -0.03436907, -0.03797608]])])

In [46]:
data = background.values

In [47]:
data = dataset.drop(columns=targets_only)
data

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_warning
0,Very High,Low,Low,Very High,High,Medium,High,Very High,True,march,spring,saturday,False
1,High,Low,High,High,Low,Low,High,High,False,january,winter,thursday,False
2,Very High,High,High,Very High,Medium,Medium,Very High,Very High,False,january,winter,friday,False
3,Low,Low,Low,Low,Medium,Low,Low,Medium,False,february,winter,friday,False
4,Very High,Low,Very High,Very High,Low,Low,Low,Very High,True,december,winter,saturday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41668,Low,Low,Very High,Low,Low,Low,Low,Medium,True,january,winter,saturday,False
41669,Medium,Medium,Very High,Medium,High,Medium,Medium,High,False,february,winter,tuesday,False
41670,Medium,Very High,High,Medium,Low,High,Low,High,False,april,spring,tuesday,False
41671,Low,Low,Very High,Low,High,High,Medium,Low,True,january,winter,saturday,False


In [48]:
rand_state = 10
observation = data.sample(5, random_state=rand_state).iloc[:,:-1].values
real_target = data.sample(5, random_state=rand_state).iloc[:,-1].values
real_target

array([False, False, False,  True, False])

mimic predict method

In [49]:
def find_outcome(data, observation):
    # Iterate through each record in the data
    # print(len(data.values))
    for record in data.values:
        # print('obs', observation)
        if (record[:-1] == observation).all():
            print('Found!')
            print('Record: ', record)
            print('Observation: ', observation)
            return record[-1]  # Return the last item (True/False) if there's a match
    return None  # Return None if  no match is found

# for obs in observation:
# print(observation)
for obs in observation:
    outcome = find_outcome(data, obs)
    print('outcome', outcome)
    # print("The outcome for observation", obs, "is:", outcome)


Found!
Record:  ['Low' 'Medium' 'Very High' 'Low' 'Very High' 'High' 'High' 'Medium' False
 'june' 'summer' 'monday' False]
Observation:  ['Low' 'Medium' 'Very High' 'Low' 'Very High' 'High' 'High' 'Medium' False
 'june' 'summer' 'monday']
outcome False
Found!
Record:  ['Very High' 'Very High' 'High' 'Very High' 'Medium' 'Very High' 'Low'
 'Medium' False 'march' 'spring' 'monday' False]
Observation:  ['Very High' 'Very High' 'High' 'Very High' 'Medium' 'Very High' 'Low'
 'Medium' False 'march' 'spring' 'monday']
outcome False
Found!
Record:  ['Medium' 'High' 'High' 'Medium' 'Low' 'Very High' 'Low' 'Medium' False
 'august' 'summer' 'friday' False]
Observation:  ['Medium' 'High' 'High' 'Medium' 'Low' 'Very High' 'Low' 'Medium' False
 'august' 'summer' 'friday']
outcome False
Found!
Record:  ['Very High' 'High' 'Low' 'Very High' 'Very High' 'High' 'Very High'
 'Very High' False 'october' 'autumn' 'tuesday' True]
Observation:  ['Very High' 'High' 'Low' 'Very High' 'Very High' 'High' 'Very 

old

In [204]:
def find_outcome(data, observation):
    # Iterate through each record in the data
    for record in data:
        if (record[:-1] == observation.values[0][:-1]).all():
            return record[-1]  # Return the last item (True/False) if there's a match
    return None  # Return None if  no match is found

outcome = find_outcome(data, observation)
print("The outcome for observation", observation, "is:", outcome)


AttributeError: 'numpy.ndarray' object has no attribute 'values'